In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab
import seaborn as sns
import pmdarima as pm
import itertools 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from pmdarima import auto_arima
color_pal = sns.color_palette()
from flask import Flask, request, jsonify
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings 
warnings.filterwarnings('ignore')


# LOADING AND VIEWING DATA

In [ ]:
# Loading the dataset using def fuction and making sure of few things like:
#parse_dates enables pandas to understand that it is a date and not string
def gb_carbon():
    # Read the CSV file and parse the dates
    data = pd.read_csv('gb_carbon_intensity.csv', parse_dates=True)
    data.set_index('datetime', inplace=True)
    return data

CO2 =gb_carbon()
print(CO2.head()) #checking the first data

In [ ]:
print(CO2.tail()) #checking the last five data

In [ ]:
print(CO2.info()) #Checking the information of each columns

# DATA CLEANING

In [ ]:
#The datetime shows a timespam of 30minutes interval and I will split it based on that
# 30min, 1 hour, day_of_week, month, quarter, year, day_of_year.
def create_timespam(CO2):
    if not pd.api.types.is_datetime64_any_dtype(CO2.index):
        CO2.index = pd.to_datetime(CO2.index)
        
    CO2['30mi'] = CO2.index.floor('30T').time # this is to split it into 30min but it will be an object instead of a float
    CO2['30min'] = CO2['30mi'].apply(lambda x: x.hour * 60 + x.minute + x.second / 60) #For the 30mins to be a float
    CO2.drop(columns=['30mi'], inplace=True)
    CO2['hour'] = CO2.index.hour
    CO2['dayofweek'] = CO2.index.dayofweek
    CO2['month'] = CO2.index.month
    CO2['quarter'] = CO2.index.quarter
    CO2['year'] = CO2.index.year
    CO2['dayofyear'] = CO2.index.dayofyear
    # Every other time split will have the right datatype 
    return CO2
CO2 = create_timespam(CO2)
CO2.head()

In [ ]:
#I have to drop, forecast and the index columns  
CO2.drop(columns=['index', 'forecast'], inplace=True)
# print(CO2.head())
CO2.head()

In [ ]:
CO2.tail()

In [ ]:
#Seeing that some rows has 'NaN' I will remove the empyt rows 'NaN'
#These NaNs rows was created but had not yet been retrived when the data was downloaded from the website
CO2.dropna(inplace=True)
CO2.tail()

In [ ]:
# I want to check if there is still NaN's in the dataset
CO2_has_nan = CO2.isnull().values.any()
print(f"Does the dataset have NaN values? {CO2_has_nan}") # to reconfirm if there is still any 'NaNs'

# PERFORM SPECIFIC ATA ANALYSIS (SDA)

In [ ]:
fig, ax = plt.subplots(figsize=(30,8))
sns.boxplot(data=CO2, x='30min', y='actual')
ax.set_title('CO2 Emission by 30minutes')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
sns.boxplot(data=CO2, x='hour', y='actual')
ax.set_title('CO2 Emission by hour')
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(data=CO2, x='dayofweek', y='actual')
ax.set_title('CO2 Emission by days of the week')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(data=CO2, x='month', y='actual')
ax.set_title('CO2 Emission by month')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(data=CO2, x='quarter', y='actual')
ax.set_title('CO2 Emission by quarter')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(data=CO2, x='year', y='actual')
ax.set_title('CO2 Emission by year')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(28,10))
sns.boxplot(data=CO2, x='dayofyear', y='actual')
ax.set_title('CO2 Emission by days of the year')
plt.show()

# TO DETERMINE IF THE DATA IS STATIONARY OR NOT

In [ ]:
# METHOD 1: using visualization to see if the data (the 'actual' column which is my focus) is stationary or not

plt.rcParams['font.size'] = 15
CO2['actual'].plot(figsize=(15,5), color=color_pal[0], title = 'Great Britain CO2 Current emission');
plt.show()

In [ ]:
#METHOD 2: using adfuller to determine if the 'actual' column is stationary or not

def stationarity_check(actual):
    """
    Check if the 'actual' column is stationary  using function.
    """
    # Performed the Augmented Dickey-Fuller test
    result = adfuller(actual, autolag = 'AIC')
    print('1. ADF :', result[0])
    print('2. P_value: ', result[1])
    print('3. Num. of Lags: ', result[2])
    print('4. Num. of observation: ', result[3])
    print('5. Critical value :')
    for key, values in result[4].items():
        print('\t', key, ': ', values)
    
    # To determine from the result of p-value if 'actual' column is stationary or not
    if result[1] < 0.05:
        print("The actual column is stationary (Therefore, reject null hypothesis)")
    else:
        print("The actual column is non-stationary (Therefore, fail to reject null hypothesis)")
        
stationarity_check(CO2['actual'])


# CREATING TIME SERIES MODELS

### SPLIT THE DATA INTO TRAIN AND TEST

In [ ]:
#Time Series Cross Validation to split the data
#The goal is to get 30-minute intervals in the last 3 months
tss = TimeSeriesSplit(n_splits=5, test_size=4320, gap=48) #The split is done with 30 min interval to predict the last 3 months
CO2 = CO2.sort_index()

In [ ]:
#This to get my train and validation index using the timeseriessplit i used in the previous cell
for arima_train_idex, arima_vali_idex in tss.split(CO2):
    break 

In [ ]:
#My train index output
arima_train_idex

In [ ]:
#my validation(test) index output
arima_vali_idex

In [ ]:
#The goal of this code is to use the 'forecast' column already on the data and compare it with my forecast
# The 'forecast' column is the result from the website i downloaded my data. I want to see which prediction is most accurate  
#TimeSeriesSplit
tss = TimeSeriesSplit(n_splits=5, test_size=4320, gap=48)
CO2 = CO2.sort_index()

for arima_train_idex, arima_vali_idex in tss.split(CO2):
    break 

# Visualizing my TimeSeriesSplit
fig, axs = plt.subplots(5, 1, figsize=(15, 15), sharex=True)

cell = 0
splits = []
for arima_train_index, arima_test_index in tss.split(CO2):
    arima_train = CO2.iloc[arima_train_index]
    arima_test = CO2.iloc[arima_test_index]
    splits.append((arima_train, arima_test)) 
    
    arima_train['actual'].plot(ax=axs[cell], label='Training Set', title=f'Data Train/Test Split cell {cell}')
    arima_test['actual'].plot(ax=axs[cell], label='Test Set')
    axs[cell].axvline(arima_test.index.min(), color='black', ls='--')
    cell += 1

plt.show()


In [ ]:
#This helps me to get all the possible comnibation in the list element
p = range(0, 2)
d = range(0, 2)
q = range(0, 2)

pdq = list(itertools.product(p, d, q))


seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print(seasonal_pdq)

In [ ]:

# Placeholder for my data
CO2 = CO2['actual']

# search over the parameters above
for param in pdq:
    for param_sea in seasonal_pdq:
        try:
            # my model section
            model = sm.tsa.statespace.SARIMAX(CO2,
                                             order=param,
                                             seasonal_order=param_sea,
                                             enforce_stationarity=False,
                                             enforce_invertibility=False)
            # model fit
            results = model.fit()
            # results output
            print(f'SARIMAX{param}x{param_sea} - AIC: {results.aic}')
        except Exception as e:
            # Print the exception if any occurs
            print(f'Error for SARIMAX{param}x{param_sea}: {e}')
            continue


FITTING THE BEST PARAMETER INTO THE MODEL

In [ ]:
# Based on the training above, the best  is SARIMAX(1, 1, 1)x(1, 0, 1, 12) - AIC: 708999.2968599194
# Define the model with the identified best parameters
arima_best_model = sm.tsa.statespace.SARIMAX(CO2,
                                             order=(1, 1, 1),
                                             seasonal_order=(1, 0, 1, 12),
                                             enforce_stationarity=False,
                                             enforce_invertibility=False)

# Fit the model
results = arima_best_model.fit()

# Print the summary of the model
print(results.summary())

In [ ]:
#diagnostics to see how the data plays out
results.plot_diagnostics(figsize=(15,12))
plt.show()

In [ ]:
# Checking for confidence interval (lower and upper values )
# May is the starting of my 3 month. my data ends in July based on the data
# so, i want to predict from May... 
pred =results.get_prediction(start=pd.to_datetime('2024-05-07 00:00:00'), dynamic=False) 
pred_ci =pred.conf_int()
pred_ci

In [ ]:
CO2.head()

In [ ]:
# I want to predict the last 3 months and compare it with actual value to see the accuracy
start_date = '2024-05-07' #start of my testing part

# Get in-sample predictions from the test date onwards
pred = results.get_prediction(start=start_date, dynamic=False)

# Extract the predicted mean values
y_predicted = pred.predicted_mean

# Actual values from the starting date onwards
y_truth = CO2[start_date:].copy()

# Combining actual and predicted values to see side by side comparison
CO2_comparison = pd.DataFrame({
    'actual': y_truth,
    'predicted': y_predicted
})

# Display the comparison
print(CO2_comparison)


In [ ]:
# Since my data has multiple columns, including 'forecast' and 'actual', i will focus on these
# I want to predict the last 3 months and compare it with actual value to see the accuracy
start_date = '2024-05-07' #start of my testing part

# Get predictions starting from the defined start date
pred = results.get_prediction(start=start_date, dynamic=False)

# Extract the predicted mean values
y_predicted = pred.predicted_mean

# Extract actual values from the start date onwards
y_truth = CO2['actual'][start_date:].copy()

# Extract the forecast values from the start date onwards
y_forecast = CO2['forecast'][start_date:].copy()

# Align indices of the series
y_predicted = y_predicted.reindex(y_truth.index)
y_forecast = y_forecast.reindex(y_truth.index)

# Combine actual, forecast, and predicted values into a DataFrame
CO2_comparison = pd.DataFrame({
    'actual': y_truth,
    'forecast': y_forecast,
    'predicted': y_predicted
})

# Display the comparison
print(CO2_comparison)


In [ ]:
# To visualize Twelve-weeks period to see how well the model performed
start_date = '2024-05-07'

# Filter my data for the Twelve-weeks period
end_date = pd.to_datetime(start_date) + pd.DateOffset(weeks=12)
Twelve_weeks_CO2 = CO2_comparison[start_date:end_date]

# Plotting the actual vs predicted values for Twelve weeks
plt.figure(figsize=(20, 10))
plt.plot(Twelve_weeks_CO2.index, Twelve_weeks_CO2['actual'], label='Actual CO2')
plt.plot(Twelve_weeks_CO2.index, Twelve_weeks_CO2['predicted'], label='Predicted CO2', linestyle='--')
plt.title('Actual vs Predicted CO2 Emission (12 Weeks)')
plt.xlabel('Date')
plt.ylabel('CO2 Emission values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Extract actual and predicted values to enable me calculate the accurancy
y_actual = CO2_comparison['actual']
y_predicted = CO2_comparison['predicted']

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_actual, y_predicted)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_actual, y_predicted)

# Calculate R-squared (R²)
r2 = r2_score(y_actual, y_predicted)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Print the metrics
print(f"Predicted Mean Squared Error (MSE): {mse:.4f}")
print(f"Predicted Mean Absolute Error (MAE): {mae:.4f}")
print(f"Predicted R-squared (R²): {r2:.4f}")
print(f"Predicted Root Mean Squared Error (RMSE): {rmse:.4f}")


In [ ]:
#I want to compare the forecast that I saw when i downloaded the data aganist my forecast to see which is better
#I will call this baseline 2 while the actual result of CO2 will be called baseline 1
# the baseline is always the same across the both model so there will be no need to check for Prophet model.
# based on this, the result gotten here will be used for compare aganist the prophet model 
# To get my baseline from the forecast that was already in the dataset

# Get the last observed value in the training set
last_observed_value = CO2[start_date].iloc[-1]

# Create a baseline prediction for the same length as the test period
baseline_pred = [last_observed_value] * len(y_actual)


In [ ]:
print(CO2_comparison.columns)

In [ ]:
# Extract the forecast and actual values
y_forecast = CO2_comparison['forecast']
y_actual = CO2_comparison['actual']

# Calculate the error metrics for the baseline forecast
mse_forecast = mean_squared_error(y_actual, y_forecast)
mae_forecast = mean_absolute_error(y_actual, y_forecast)
r2_forecast = r2_score(y_actual, y_forecast)
rmse_forecast = np.sqrt(mse_forecast)

# Print the ARIMA metrics (i have already written this code earlier)
print(f"Predicted Mean Squared Error (MSE): {mse:.4f}")
print(f"Predicted Mean Absolute Error (MAE): {mae:.4f}")
print(f"Predicted R-squared (R²): {r2:.4f}")
print(f"Predicted Root Mean Squared Error (RMSE): {rmse:.4f}")


# Print the baseline metrics
print(f"Baseline Forecast - Mean Squared Error (MSE): {mse_forecast:.4f}")
print(f"Baseline Forecast - Mean Absolute Error (MAE): {mae_forecast:.4f}")
print(f"Baseline Forecast - R-squared (R²): {r2_forecast:.4f}")
print(f"Baseline Forecast - Root Mean Squared Error (RMSE): {rmse_forecast:.4f}")


# The END